## Part 1

In [2]:
def use_map(ranges: list[list[int]], source: int):
    for dest_start, source_start, length in ranges:
         if source_start <= source < source_start + length:
              return dest_start + (source - source_start)
    return source

In [3]:
def parse_map(text: str):
    return [[int(x) for x in s.split()] for s in text.split('\n')[1:]]

In [4]:
def parse_almanac(almanac: str):
    sections = almanac.split('\n\n')
    seeds = [int(x) for x in sections[0].split(':')[1].split()]
    ranges = [parse_map(t) for t in sections[1:]]
    return seeds, ranges

In [5]:
with open('input.txt') as f:
    almanac = f.read()

In [6]:
seeds, all_ranges = parse_almanac(almanac)


In [7]:
locations = []
for seed in seeds:
    source = seed
    for ranges in all_ranges:
        dest = use_map(ranges, source)
        source = dest
    locations.append(dest)
print(min(locations))

278755257


## Part 2

In [8]:
import re
def parse_almanac_2(almanac: str):
    sections = almanac.split('\n\n')
    pattern = re.compile(r'\d+ \d+')
    seed_pairs = [(int(x), int(y)) for (x, y) in [pair.split() for pair in re.findall(pattern, sections[0])]]
    mapping_sets = [parse_map(t) for t in sections[1:]]
    return seed_pairs, mapping_sets

In [9]:
seed_pairs, mapping_sets = parse_almanac_2(almanac)

You should reverse it. Convert a location into a seed. And when a location matches an existing seed, you've got the lowest possible location as your iterator.

Also: instead of making a list with all seeds, make a function that uses comparison operators to check if a seed is in the list

Taking the first seeds as lists with the first and the last seed of the range. Obviously these ranges will have to be eventually divided into more ranges. And at the end your solution will be the smallest number in the final ranges

- https://mathematico.netlify.app/blog/aoc-23-5
- https://github.com/PetchyAL/AoC2023/blob/main/solutions/day05/day5.py 
- https://github.com/xHyroM/aoc/tree/main/2023/05 
- https://github.com/RomainPierre7/Advent-of-Code/tree/main/2023/day05 

In [14]:
class Domain():
    def __init__(self, start: float, end: float):
        try:
            assert start < end
        except AssertionError:
            print(start, end)
            raise
        self.start = start
        self.end = end
    def __repr__(self):
        return f'domain(start: {self.start}, len: {self.end})'
    
class Mapping():
    def __init__(self, start: float, end: float, shift: float=0):
        try:
            assert start < end
        except AssertionError:
            print(start, end)
            raise
        self.start = start
        self.end = end
        self.shift = shift
    def __repr__(self):
        return f'mapping(src: {self.start}, end: {self.end}, shift: {self.shift})'

In [11]:
def get_identities(mappings: list[Mapping]):
    sorted_mappings = sorted(mappings, key=lambda x: x.start)
    lower = Mapping(float('-inf'), sorted_mappings[0].start - 1)
    upper = Mapping(sorted_mappings[-1].end, float('inf')) ## maybe should be from sorted_mappings[-1].end + 1

    identity_mappings = [lower, upper]

    for i in range(len(sorted_mappings) - 1):
        lower_end, upper_start = sorted_mappings[i].end, sorted_mappings[i + 1].start
        if lower_end < upper_start:
            identity_mappings.append(Mapping(lower_end, upper_start))
    
    return identity_mappings

In [16]:
def prop_one_level(domains: list[Domain], filled_mappings: list[Mapping]):
    ranges: list[Domain] = []
    for domain in domains:
        for mapping in filled_mappings:
            if not (mapping.start >= domain.end or mapping.end <= domain.start): # think of domains and mappings as being intervals of the format [start, end)
                slice_start = max(domain.start, mapping.start) 
                slice_end = min(domain.end, mapping.end)
                ranges.append(Domain(slice_start + mapping.shift, slice_end + mapping.shift))
    return ranges

In [19]:
domains = [Domain(seed[0], seed[0] + seed[1]) for seed in seed_pairs]
for mapping_set in mapping_sets:
    mappings = [Mapping(mapping[1], mapping[1] + mapping[2], mapping[0] - mapping[1]) for mapping in mapping_set]
    identity_mappings = get_identities(mappings)
    filled_mappings = mappings + identity_mappings
    ranges = prop_one_level(domains, filled_mappings)
    domains = ranges
print(min([domain.start for domain in domains]))

26829166
